# Rolling 30 Days Retention Rate
### of Resellers in a Dropshipping Company

In a dropshipping company, once a resellers places an order, she is considered retaind for the next 29 days. If she maked more orders in this 29 days interval, she is retained, unless she is counted as churned.

In this report, day 1 is the day each reseller place their forst order (also week 1 and month 1). To calculate the retention rate for each date, I count the number of retained resellers in the [date-29, date] interval. 

In [ ]:
with base AS (SELECT
    `orders`.`ID` AS `OrderID`,
    `orders`.`UserID` AS `UserID`,
    `orders`.`CreatedAt` AS `OrderDate`,
    `orders`.`FakeDelete` AS `FakeDelete`,
    `Users`.`TotalOrders` AS `TotalOrders`,
    `Users`.`CreatedAt` AS `JoinDate`,
    `Vw Orders Is Net`.`isGross` AS `isGross`,
    `Vw Users First Order`.`FirstOrderAt` AS `ActivationDate`,
    DATEDIFF(`orders`.`CreatedAt`, `Vw Users First Order`.`FirstOrderAt`) + 1 AS `Day`,
    CEILING((DATEDIFF(`orders`.`CreatedAt`, `Vw Users First Order`.`FirstOrderAt`)+1)/7) as `Week`,
    DATEDIFF( `orders`.`CreatedAt`, coalesce( LAG(`orders`.`CreatedAt`) OVER ( PARTITION BY UserID ORDER BY `orders`.`CreatedAt`), 0)
      ) AS DistanceFromPreviousOrder,
    ROW_NUMBER() OVER (PARTITION BY UserID ORDER BY orders.CreatedAt DESC ) AS OrderOrder,
    -- DATE_ADD(Day, INTERVAL 29 DAY) AS `RetentionDue`
    DATEDIFF(`orders`.`CreatedAt`, `Vw Users First Order`.`FirstOrderAt`) + 30 AS `RetentionDue`,
    DATEDIFF(`orders`.`CreatedAt`, `Vw Users First Order`.`FirstOrderAt`) - 29 AS `Start`
FROM
    `orders`
    INNER JOIN `users` AS `Users` ON `orders`.`UserID` = `Users`.`ID`
    INNER JOIN `vw_orders_is_net` AS `Vw Orders Is Net` ON `orders`.`ID` = `Vw Orders Is Net`.`OrderID`
    INNER JOIN `vw_users_first_order` AS `Vw Users First Order` ON `Users`.`ID` = `Vw Users First Order`.`UserID`
WHERE
   (`Vw Orders Is Net`.`isGross` = 1)
   AND (`orders`.`FakeDelete` = 0)
ORDER BY `orders`.`UserID`),

myOrder AS (SELECT Day, UserID, max(RetentionDue) AS RetentionDue FROM base GROUP BY Day, UserID ORDER BY Day),

myDays AS (SELECT DISTINCT Day FROM myOrder ORDER BY Day),

myJoin AS (SELECT myDays.Day AS DDay, myOrder.Day AS ODay, UserID
FROM myDays, myOrder
WHERE myOrder.Day BETWEEN (myDays.Day-29) AND myDays.Day),

myFinal AS (SELECT DDay AS Day, COUNT(DISTINCT ODay) AS Duration, MIN(ODay) AS StartDay, MAX(ODay) AS EndDay, COUNT(DISTINCT UserID) AS ActivesInInterval
FROM  myJoin
GROUP BY Day),

allActivated AS (SELECT MAX(ActivesInInterval) AS AllActivation FROM myFinal)

SELECT Day, Duration, StartDay, EndDay, ActivesInInterval, ActivesInInterval/AllActivation AS 'PercentActivesInInterval'
FROM  myFinal, allActivated
WHERE Day > 30;